In [2]:
import os 
import shutil
import cv2
import random

In [12]:
## GATHER ALL TEXTS AND PHOTOS TOGETHER
JPGDIR = 'contents/images'
TXTDIR = 'contents/merged_ann'
DSTDIR = "contents/mixed"
if not os.path.isdir(DSTDIR):
    os.mkdir(DSTDIR)


txtList = []
for dirpath, dirnames, filenames in os.walk(TXTDIR):
    for filename in filenames:
        if not filename.endswith(".txt"):
            continue
        fullpath = os.path.join(dirpath, filename)
        shutil.copy(fullpath, DSTDIR)
# print(txtList)


jpgList = []
for dirpath, dirnames, filenames in os.walk(JPGDIR):
    for filename in filenames:
        if not filename.endswith(".jpg"):
            continue
        fullpath = os.path.join(dirpath,filename)
        shutil.copy(fullpath, DSTDIR)

In [ ]:
BEGINDIR1 = "ann1"
BEGINDIR2 = "ann2"
MERGEDIR = "merged_ann"
list1 = os.listdir(BEGINDIR1)
list2 = os.listdir(BEGINDIR2)
#print(list1)
list1.sort() 
list2.sort()
if not (os.path.isdir(MERGEDIR)):
    os.mkdir(MERGEDIR)
def merger(src1, src2):
    with open(src1, "a") as f:
        with open(src2, "r") as f2:
            #f.write("\n")
            f.write(f2.read())
    shutil.copy(src1, MERGEDIR)



i = 0
j = 0
while i < len(list1) and j < len(list2):
    if(list1[i] == list2[j] ):
        merger(f"{BEGINDIR1}/{list1[i]}" , f"{BEGINDIR2}/{list2[j]}")
        i+=1
        j+=1
    else:
        if list1[i] < list2[j]:
            shutil.copy(f"{BEGINDIR1}/{list1[i]}", MERGEDIR)
            i+=1
        else:
            shutil.copy(f"{BEGINDIR2}/{list2[j]}", MERGEDIR)
            j+=1
while i < len(list1):
    shutil.copy(f"{BEGINDIR1}/{list1[i]}", MERGEDIR)
    i+=1

while j < len(list2):
    shutil.copy(f"{BEGINDIR2}/{list2[j]}", MERGEDIR)
    j+=1


In [13]:
## MAKE ALL THE PAIRS. WEED OUT THE UNPAIRED JPG FILES
UNPAIREDDIR = "contents/unpaired"
if not os.path.isdir(UNPAIREDDIR):
    os.mkdir(UNPAIREDDIR)

MIXDIR = 'contents/mixed'

txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]
# print(txtList)

jpgList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    jpgList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".jpg")]

# print(jpgList)

for photo in jpgList:
    txtname:str = photo[:-4] + ".txt"
    # print(txtname)
    
    if not (txtname in txtList):
        # not paired
        just_file_name = photo.split("/")[-1]
        
        os.rename(photo, f"contents/unpaired/{just_file_name}")
        # shutil.copy(photo, f"contents/unpaired")
        # print(photo)

In [14]:
## CHECK FOR EMPTY LINES

def contains_blank_line(file_path):
    with open(file_path, "r") as file:
        content = file.read()
        content = content.split("\n")
        while not all(content):
            content = content[:-1]
        return content

def strip_blank_line(file_path):
    content = contains_blank_line(file_path)
    with open(file_path,"w") as test:
        for index, line in enumerate(content):
            test.write(line)
            if not index == (len(content)-1):
                test.write("\n")

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]

for file_path in txtList:
    strip_blank_line(file_path=file_path)

In [11]:
## REPLACE STARTING NUMBERS IF NOT ZERO(0)

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file}' for file in filenames if file.endswith(".txt")]


for file in txtList:
    lines = 0
    line_split: str
    with open(file, "r") as file_inside:
        content = file_inside.read()
        line_split = content.split(' ')

    try:
        if not int(line_split[0]) == 0:
            print(line_split, file)
            with open(file, "w") as f:
                text_to_write = f"0 {line_split[1]} {line_split[2]} {line_split[3]} {line_split[4]}"
                f.write(text_to_write)
    except ValueError:
        print(line_split, file)
        if len(line_split) < 5:
            os.remove(file)
        else: break

[''] contents/mixed/photo2095.txt
[''] contents/mixed/photo2099.txt
[''] contents/mixed/photo3871.txt
[''] contents/mixed/photo3981.txt
[''] contents/mixed/photo5427.txt
['1', '0.650521', '0.216204', '0.463542', '0.423148'] contents/mixed/photo5859.txt
['1', '0.160156', '0.551389', '0.061979', '0.023148'] contents/mixed/photo586.txt
['1', '0.652344', '0.233333', '0.436979', '0.461111'] contents/mixed/photo5860.txt
['1', '0.642708', '0.269907', '0.432292', '0.530556'] contents/mixed/photo5861.txt
['1', '0.664062', '0.277315', '0.455208', '0.550926'] contents/mixed/photo5862.txt
['1', '0.683073', '0.296296', '0.461979', '0.566667'] contents/mixed/photo5863.txt
['1', '0.665625', '0.266667', '0.451042', '0.525926'] contents/mixed/photo5864.txt
['1', '0.647396', '0.280556', '0.444792', '0.551852'] contents/mixed/photo5865.txt
['1', '0.643229', '0.280556', '0.445833', '0.544444'] contents/mixed/photo5866.txt
['1', '0.646615', '0.313889', '0.453646', '0.603704'] contents/mixed/photo5867.txt
[

In [15]:
## SPLIT ALL PAIRS ACCORDING TO A RATIO

MIXDIR = "contents/mixed"
txtList = []
for dirpath, dirnames, filenames in os.walk(MIXDIR):
    txtList += [f'{dirpath}/{file[:-4]}' for file in filenames if file.endswith(".txt")]

SPLIT_RATIO = 1 / 4
SPLIT_PROBABILITY = SPLIT_RATIO / (SPLIT_RATIO + 1)
MAX_TEST_SET_SIZE = len(txtList) * SPLIT_PROBABILITY



TESTDIR = "contents/val"
if not os.path.isdir(TESTDIR):
    os.mkdir(TESTDIR)
    os.mkdir(TESTDIR + "/labels")
    os.mkdir(TESTDIR + "/images")

TRAINDIR = "contents/train"
if not os.path.isdir(TRAINDIR):
    os.mkdir(TRAINDIR)
    os.mkdir(TRAINDIR + "/labels")
    os.mkdir(TRAINDIR + "/images")

test_set_size = 0
for paired_name in txtList:

    if random.random() < SPLIT_PROBABILITY and test_set_size <= MAX_TEST_SET_SIZE:
        shutil.copy(f"{paired_name}.txt",f"{TESTDIR}/labels/")
        shutil.copy(f"{paired_name}.jpg",f"{TESTDIR}/images/")
        test_set_size += 1
    else:
        shutil.copy(f"{paired_name}.txt",f"{TRAINDIR}/labels/")
        shutil.copy(f"{paired_name}.jpg",f"{TRAINDIR}/images/")

FileNotFoundError: [Errno 2] No such file or directory: 'contents/mixed/classes.jpg'

In [ ]:
## GENERATE TEXT FILES FOR THE TRAINING AND TESTING DATA SETS
TESTDIR = "contents/test"
TRAINDIR = "contents/obj"

allTexts = []
for dirpath, dirnames, filenames in os.walk(TESTDIR):
    allTexts += [f"data/test/{textFile}" for textFile in filenames if str(
        textFile[-4:]) == ".jpg"]
print(allTexts)

with open("test.txt", "a") as f:

    for txtFile in allTexts:
        f.write(txtFile)
        f.write("\n")

allTexts = []
for dirpath, dirnames, filenames in os.walk(TRAINDIR):
    allTexts += [f"data/obj/{textFile}" for textFile in filenames if str(
        textFile[-4:]) == ".jpg"]
print(allTexts)

with open("train.txt", "a") as f:

    for txtFile in allTexts:
        f.write(txtFile)
        f.write("\n")

In [40]:
## ARCHIVE OBJ AND TEST, PREPARE FOR UPLOAD
UPLOADDIR = "upload"

if not os.path.isdir(UPLOADDIR):
    os.mkdir(UPLOADDIR)

# shutil.make_archive("upload/obj", "zip", "contents/obj")
shutil.make_archive("upload/test","zip", "contents/test")
os.rename("test.txt", "upload/test.txt")
os.rename("train.txt", "upload/train.txt")
